In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports
import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler
from src.beam import Algorithm
from src.beam import LinearNet

from src.beam import DataTensor
from src.beam.utils import is_notebook
from cifar10_example import cifar10_algorithm_generator, Cifar10Network

from ray.tune.suggest.hebo import HEBOSearch

2022-06-14 12:51:13,043	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:13065
2022-06-14 12:51:14,796	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-06-14 12:51:15,366	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_b281c113605b1358.zip' (0.71MiB) to Ray cluster...
2022-06-14 12:51:15,389	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_b281c113605b1358.zip'.


In [2]:
path_to_data = '/localdata/elads/data/datasets/cifar10'
root_dir = '/localdata/elads/data/cifar10'

# path_to_data = '/home/shared/data/dataset/cifar10'
# root_dir = '/home/shared/data/results/cifar10'

## Training with a single worker

In [7]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=0.01 --batch-size=1024",
                      "--n-epochs=6 --epoch-length-train=1000000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=1e-5 --beta1=0.9 --beta2=0.9", path_to_data=path_to_data)

In [3]:
# here you put all actions which are performed only once before initializing the workers
# for example, setting running arguments and experiment:

args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=0.01 --batch-size=1024",
                      "--n-epochs=6 --epoch-length-train=1000000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=1e-5 --beta1=0.9 --beta2=0.9", path_to_data=path_to_data)

experiment = Experiment(args)

2022-06-14 06:19:35.973 | INFO     | src.beam.experiment:__init__:99 - beam project: cifar10
2022-06-14 06:19:35.975 | INFO     | src.beam.experiment:__init__:100 - Experiment Hyperparameters
2022-06-14 06:19:35.976 | INFO     | src.beam.experiment:__init__:104 - project_name: cifar10
2022-06-14 06:19:35.976 | INFO     | src.beam.experiment:__init__:104 - identifier: debug
2022-06-14 06:19:35.977 | INFO     | src.beam.experiment:__init__:104 - algorithm: CIFAR10Algorithm
2022-06-14 06:19:35.978 | INFO     | src.beam.experiment:__init__:104 - mp_port: None
2022-06-14 06:19:35.979 | INFO     | src.beam.experiment:__init__:104 - root_dir: /localdata/elads/data/cifar10
2022-06-14 06:19:35.979 | INFO     | src.beam.experiment:__init__:104 - reload: False
2022-06-14 06:19:35.980 | INFO     | src.beam.experiment:__init__:104 - resume: -1
2022-06-14 06:19:35.981 | INFO     | src.beam.experiment:__init__:104 - override: False
2022-06-14 06:19:35.981 | INFO     | src.beam.experiment:__init__:104

## Train with single or multiple workers

In [4]:
alg = experiment(cifar10_algorithm_generator)

2022-06-14 06:19:36.782 | INFO     | src.beam.experiment:run:454 - Single worker mode
2022-06-14 06:19:36.783 | INFO     | src.beam.experiment:run_worker:44 - Worker: 1/1 is running...


train:   0%|          | 0/975 [00:00<?, ?it/s]

test:   0%|          | 0/9 [00:00<?, ?it/s]


2022-06-14 06:20:51.928 | INFO     | src.beam.experiment:save_model_results:308 - Finished epoch 1/6:
2022-06-14 06:20:51.933 | INFO     | src.beam.experiment:log_data:346 - train:
2022-06-14 06:20:51.934 | INFO     | src.beam.experiment:log_data:350 - loss 528.213 	|
2022-06-14 06:20:51.935 | INFO     | src.beam.experiment:log_data:350 - acc 0.81942 	|
2022-06-14 06:20:51.936 | INFO     | src.beam.experiment:log_data:350 - lr 0.01 	|
2022-06-14 06:20:51.937 | INFO     | src.beam.experiment:log_data:346 - test:
2022-06-14 06:20:51.937 | INFO     | src.beam.experiment:log_data:350 - loss 530.667 	|
2022-06-14 06:20:51.938 | INFO     | src.beam.experiment:log_data:350 - acc 0.832899 	|


train:   0%|          | 0/975 [00:00<?, ?it/s]

...

test:   0%|          | 0/9 [00:00<?, ?it/s]


2022-06-14 06:25:26.106 | INFO     | src.beam.experiment:save_model_results:308 - Finished epoch 5/6:
2022-06-14 06:25:26.111 | INFO     | src.beam.experiment:log_data:346 - train:
2022-06-14 06:25:26.111 | INFO     | src.beam.experiment:log_data:350 - loss 87.093 	|
2022-06-14 06:25:26.112 | INFO     | src.beam.experiment:log_data:350 - acc 0.971361 	|
2022-06-14 06:25:26.113 | INFO     | src.beam.experiment:log_data:350 - lr 0.0001 	|
2022-06-14 06:25:26.114 | INFO     | src.beam.experiment:log_data:346 - test:
2022-06-14 06:25:26.114 | INFO     | src.beam.experiment:log_data:350 - loss 435.583 	|
2022-06-14 06:25:26.115 | INFO     | src.beam.experiment:log_data:350 - acc 0.882487 	|


train:   0%|          | 0/975 [00:00<?, ?it/s]

test:   0%|          | 0/9 [00:00<?, ?it/s]


2022-06-14 06:26:35.101 | INFO     | src.beam.experiment:save_model_results:308 - Finished epoch 6/6:
2022-06-14 06:26:35.106 | INFO     | src.beam.experiment:log_data:346 - train:
2022-06-14 06:26:35.106 | INFO     | src.beam.experiment:log_data:350 - loss 80.7526 	|
2022-06-14 06:26:35.107 | INFO     | src.beam.experiment:log_data:350 - acc 0.973349 	|
2022-06-14 06:26:35.108 | INFO     | src.beam.experiment:log_data:350 - lr 3.16228e-05 	|
2022-06-14 06:26:35.109 | INFO     | src.beam.experiment:log_data:346 - test:
2022-06-14 06:26:35.109 | INFO     | src.beam.experiment:log_data:350 - loss 446.306 	|
2022-06-14 06:26:35.110 | INFO     | src.beam.experiment:log_data:350 - acc 0.879666 	|


In [5]:
np.mean(alg.evaluate('test')['scalar']['acc'])

test:   0%|          | 0/10 [00:00<?, ?it/s]

0.8862802922725678

In [6]:
# alg.optimizers['net'].scaler.get_scale()

## Hyperparameter search with native optuna

In [3]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --half --lr-d=0.01 --batch-size=1024",
                      "--n-epochs=4 --epoch-length-train=200000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=1e-5 --device=1 --beta1=0.9 --beta2=0.9", path_to_data=path_to_data)

study = Study(cifar10_algorithm_generator, args)

2022-06-14 12:22:13.838 | INFO     | src.beam.experiment:__init__:85 - Hyperparameter Optimization
2022-06-14 12:22:13.839 | INFO     | src.beam.experiment:__init__:86 - beam project: cifar10
2022-06-14 12:22:13.840 | INFO     | src.beam.experiment:__init__:87 - Experiment Hyperparameters
2022-06-14 12:22:13.841 | INFO     | src.beam.experiment:__init__:90 - project_name: cifar10
2022-06-14 12:22:13.842 | INFO     | src.beam.experiment:__init__:90 - identifier: debug_hp_optimization_20220614_122213
2022-06-14 12:22:13.843 | INFO     | src.beam.experiment:__init__:90 - algorithm: CIFAR10Algorithm
2022-06-14 12:22:13.844 | INFO     | src.beam.experiment:__init__:90 - mp_port: None
2022-06-14 12:22:13.845 | INFO     | src.beam.experiment:__init__:90 - root_dir: /localdata/elads/data/cifar10
2022-06-14 12:22:13.846 | INFO     | src.beam.experiment:__init__:90 - reload: False
2022-06-14 12:22:13.847 | INFO     | src.beam.experiment:__init__:90 - resume: -1
2022-06-14 12:22:13.848 | INFO    

In [58]:
def suggest(trial):
    lr = trial.suggest_loguniform("lr", 1e-3, 2e-2)
    print('My suggestion')
    print(lr)
    return {'lr_dense': lr}
    

In [59]:
study.optuna(suggest, direction='maximize', n_jobs=1, n_trials=10)

[I 2022-06-14 09:37:27,339] A new study created in memory with name: cifar10/CIFAR10Algorithm/debug_hp_optimization_20220614_093207_hp_optimization_20220614_093407_hp_optimization_20220614_093726


My suggestion
0.004812922221223897
2022-06-14 09:37:27.342 | INFO     | src.beam.experiment:__init__:169 - beam project: cifar10
2022-06-14 09:37:27.343 | INFO     | src.beam.experiment:__init__:170 - Experiment Hyperparameters
2022-06-14 09:37:27.345 | INFO     | src.beam.experiment:__init__:174 - project_name: cifar10
2022-06-14 09:37:27.345 | INFO     | src.beam.experiment:__init__:174 - identifier: debug_hp_optimization_20220614_093207_hp_optimization_20220614_093407_hp_optimization_20220614_093726
2022-06-14 09:37:27.346 | INFO     | src.beam.experiment:__init__:174 - algorithm: CIFAR10Algorithm
2022-06-14 09:37:27.347 | INFO     | src.beam.experiment:__init__:174 - mp_port: None
2022-06-14 09:37:27.348 | INFO     | src.beam.experiment:__init__:174 - root_dir: /localdata/elads/data/cifar10
2022-06-14 09:37:27.348 | INFO     | src.beam.experiment:__init__:174 - reload: False
2022-06-14 09:37:27.349 | INFO     | src.beam.experiment:__init__:174 - resume: -1
2022-06-14 09:37:27.350 |

train:   0%|          | 0/194 [00:00<?, ?it/s]

test:   0%|          | 0/9 [00:00<?, ?it/s]

train:   0%|          | 0/194 [00:00<?, ?it/s]

...

train:   0%|          | 0/194 [00:00<?, ?it/s]

test:   0%|          | 0/9 [00:00<?, ?it/s]

train:   0%|          | 0/194 [00:00<?, ?it/s]

test:   0%|          | 0/9 [00:00<?, ?it/s]

[I 2022-06-14 09:46:47,584] Trial 9 finished with value: 0.8180338541666666 and parameters: {'lr': 0.002505511913619016}. Best is trial 6 with value: 0.8381076388888888.


## Hyperparameter search with ray-tune and optuna

In [3]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --half --lr-d=0.01 --batch-size=1024",
                      "--n-epochs=5 --epoch-length-train=200000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=1e-5 --device=0 --beta1=0.9 --beta2=0.9", path_to_data=path_to_data)

study = Study(cifar10_algorithm_generator, args)

2022-06-14 12:51:22.738 | INFO     | src.beam.experiment:__init__:85 - Hyperparameter Optimization
2022-06-14 12:51:22.740 | INFO     | src.beam.experiment:__init__:86 - beam project: cifar10
2022-06-14 12:51:22.741 | INFO     | src.beam.experiment:__init__:87 - Experiment Hyperparameters
2022-06-14 12:51:22.742 | INFO     | src.beam.experiment:__init__:90 - project_name: cifar10
2022-06-14 12:51:22.743 | INFO     | src.beam.experiment:__init__:90 - identifier: debug_hp_optimization_20220614_125122
2022-06-14 12:51:22.745 | INFO     | src.beam.experiment:__init__:90 - algorithm: CIFAR10Algorithm
2022-06-14 12:51:22.746 | INFO     | src.beam.experiment:__init__:90 - mp_port: None
2022-06-14 12:51:22.748 | INFO     | src.beam.experiment:__init__:90 - root_dir: /localdata/elads/data/cifar10
2022-06-14 12:51:22.749 | INFO     | src.beam.experiment:__init__:90 - reload: False
2022-06-14 12:51:22.750 | INFO     | src.beam.experiment:__init__:90 - resume: -1
2022-06-14 12:51:22.752 | INFO    

In [ ]:
# hebo = HEBOSearch(metric="mean_accuracy", mode="max")

# analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
#                               "weight_decay": tune.loguniform(1e-6, 1e-4),
#                               "gamma": tune.loguniform(.1, .9),
#                               "dropout": tune.uniform(0, .75),
#                               "scale_down": tune.uniform(0.4, .7),
#                               "scale_up": tune.uniform(0.9, 1.2),
#                               "ratio_down": tune.uniform(0.7, .95),
#                               "ratio_up": tune.uniform(1.05, 1.4),
#                               "channels": tune.choice([128, 256, 512]),
#                               "batch_size": tune.choice([512, 1024, 2048]),},
#                        metric="mean_accuracy",
#                        max_concurrent_trials=4,
#                        resources_per_trial={"gpu": 1},
#                        mode="max",
#                        search_alg=hebo,
#                       progress_reporter=JupyterNotebookReporter(overwrite=True),
#                        num_samples=400)


analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
                              "weight_decay": tune.loguniform(1e-6, 1e-4),
                              "gamma": tune.loguniform(.1, .9),
                              "dropout": tune.uniform(0, .75),
                              "scale_down": tune.uniform(0.4, .7),
                              "scale_up": tune.uniform(0.9, 1.2),
                              "ratio_down": tune.uniform(0.7, .95),
                              "ratio_up": tune.uniform(1.05, 1.4),
                              "channels": tune.choice([128, 256, 512]),
                              "batch_size": tune.choice([512, 1024, 2048]),},
                       metric="mean_accuracy",
                       max_concurrent_trials=4,
                       resources_per_trial={"gpu": 1},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=400)


Trial name,status,loc,batch_size,channels,dropout,gamma,lr_dense,ratio_down,ratio_up,scale_down,scale_up,weight_decay,acc,iter,total time (s)
runner_tune_0d7f60d4,RUNNING,172.17.0.2:107145,512,512,0.544335,0.812667,0.00248187,0.723178,1.05638,0.680232,1.18267,4.84139e-05,0.853002,2,80.0163
runner_tune_e0652ce6,RUNNING,172.17.0.2:104311,512,512,0.552046,0.69653,0.00253653,0.819709,1.06885,0.674288,1.17596,1.77434e-05,0.880243,4,134.16
runner_tune_e6348540,RUNNING,172.17.0.2:104708,512,512,0.5467,0.688655,0.00261118,0.723102,1.08356,0.679148,1.13627,1.81614e-05,0.872533,4,135.975
runner_tune_f4929be0,RUNNING,172.17.0.2:105603,512,512,0.551451,0.709076,0.00246216,0.717842,1.07593,0.684372,1.19865,4.19013e-05,0.869757,3,104.331
runner_tune_0422009a,TERMINATED,172.17.0.2:43408,512,512,0.593296,0.320698,0.00755197,0.706525,1.23019,0.547238,1.12604,2.69908e-05,0.854955,5,166.396
runner_tune_0cf21424,TERMINATED,172.17.0.2:75507,512,128,0.524347,0.747728,0.00218162,0.731857,1.33853,0.596853,1.1485,1.0148e-06,0.810958,5,50.0915
runner_tune_12524db0,TERMINATED,172.17.0.2:28453,512,256,0.715172,0.103074,0.00139939,0.743094,1.15144,0.686847,0.923411,2.12862e-06,0.796669,5,76.6309
runner_tune_14d2f0e0,TERMINATED,172.17.0.2:91787,2048,512,0.478845,0.887274,0.00173694,0.77351,1.15507,0.698744,1.16926,1.39174e-05,0.870605,5,218.339
runner_tune_155c79be,TERMINATED,172.17.0.2:91789,2048,512,0.485775,0.899836,0.00166,0.741396,1.1582,0.695957,1.09889,1.32813e-05,0.865234,5,167.194
runner_tune_17e62cac,TERMINATED,172.17.0.2:92044,512,512,0.495628,0.422892,0.00270622,0.774116,1.26432,0.625411,1.11411,1.55294e-05,0.869449,5,168.611


Result for runner_tune_e0652ce6:
  date: 2022-06-14_13-31-18
  done: false
  experiment_id: 57948764d879419ba476a3fb1f71ebc9
  hostname: 8924131b90b2
  iterations_since_restore: 5
  mean_accuracy: 0.8845600328947368
  node_ip: 172.17.0.2
  pid: 104311
  time_since_restore: 166.15800619125366
  time_this_iter_s: 31.998493909835815
  time_total_s: 166.15800619125366
  timestamp: 1655213478
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: e0652ce6
  warmup_time: 0.0047833919525146484
  
Result for runner_tune_e0652ce6:
  date: 2022-06-14_13-31-18
  done: true
  experiment_id: 57948764d879419ba476a3fb1f71ebc9
  experiment_tag: 61_batch_size=512,channels=512,dropout=0.55205,gamma=0.69653,lr_dense=0.0025365,ratio_down=0.81971,ratio_up=1.0689,scale_down=0.67429,scale_up=1.176,weight_decay=1.7743e-05
  hostname: 8924131b90b2
  iterations_since_restore: 5
  mean_accuracy: 0.8845600328947368
  node_ip: 172.17.0.2
  pid: 104311
  time_since_restore: 166.15800619125366
  time_this_